In [1]:
import sys
import os
from PyQt5.QtWidgets import (
    QApplication, QWidget, QPushButton, QVBoxLayout, QHBoxLayout, QTableWidget,
    QTableWidgetItem, QCheckBox, QAbstractItemView, QLabel, QMessageBox
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QIcon, QFont, QColor

# For macOS-specific functionality
import platform
if platform.system() == 'Darwin':
    try:
        from AppKit import NSApp, NSWindowCollectionBehaviorCanJoinAllSpaces
        import objc
    except ImportError:
        NSApp = None

class DropArea(QLabel):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setText('Drop .xlsx files here')
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet('''
            border: 2px dashed #aaa;
            padding: 20px;
            font-size: 14px;
            background-color: #f0f0f0;
        ''')
        self.setAcceptDrops(True)
        self.parent = parent

    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls():
            # Accept if any of the dragged files are .xlsx
            for url in event.mimeData().urls():
                if url.toLocalFile().lower().endswith('.xlsx'):
                    event.accept()
                    return
        event.ignore()

    def dropEvent(self, event):
        if event.mimeData().hasUrls():
            for url in event.mimeData().urls():
                filepath = url.toLocalFile()
                if filepath.lower().endswith('.xlsx'):
                    self.parent.add_file(filepath)
            event.accept()
        else:
            event.ignore()

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Report Generator')
        self.initUI()

    def initUI(self):
        main_layout = QVBoxLayout()

        # Top bar with pin button and close button
        top_layout = QHBoxLayout()
        top_layout.addStretch()

        # Pin button
        self.pin_button = QPushButton("📌")
        self.pin_button.setCheckable(True)
        self.pin_button.clicked.connect(self.toggle_always_on_top)
        self.pin_button.setStyleSheet('''
            QPushButton {
                background-color: #e0e0e0;
                font-size: 14px;
            }
            QPushButton:checked {
                background-color: #a0a0a0;
            }
        ''')
        top_layout.addWidget(self.pin_button)

        # Close button
        self.close_button = QPushButton("Close")
        self.close_button.clicked.connect(self.close_application)
        self.close_button.setStyleSheet('''
            QPushButton {
                background-color: #ff6666;
                color: white;
                font-size: 14px;
            }
            QPushButton:hover {
                background-color: #ff3333;
            }
        ''')
        top_layout.addWidget(self.close_button)

        main_layout.addLayout(top_layout)

        # Drop area
        self.drop_area = DropArea(self)
        main_layout.addWidget(self.drop_area)

        # File table
        self.file_table = QTableWidget()
        self.file_table.setColumnCount(6)
        self.file_table.setHorizontalHeaderLabels(['Index', 'File Path', 'Manufacturer', 'Quarter', 'Option 1', 'Option 2'])
        self.file_table.setSelectionBehavior(QAbstractItemView.SelectRows)
        self.file_table.setDragDropMode(QAbstractItemView.InternalMove)
        self.file_table.setAcceptDrops(True)
        self.file_table.viewport().setAcceptDrops(True)
        self.file_table.setDragEnabled(True)
        self.file_table.setDropIndicatorShown(True)
        self.file_table.setEditTriggers(QAbstractItemView.NoEditTriggers)
        self.file_table.verticalHeader().setVisible(False)
        # Set alternate row colors
        self.file_table.setAlternatingRowColors(True)
        self.file_table.setStyleSheet('''
            QTableWidget {
                background-color: #ffffff;
                alternate-background-color: #f9f9f9;
            }
        ''')
        main_layout.addWidget(self.file_table)

        # Create Reports button
        self.create_reports_button = QPushButton('Create Reports')
        self.create_reports_button.clicked.connect(self.create_reports)
        self.create_reports_button.setStyleSheet('''
            QPushButton {
                background-color: #4CAF50;
                color: white;
                font-size: 16px;
                height: 40px;
            }
            QPushButton:hover {
                background-color: #45a049;
            }
        ''')
        main_layout.addWidget(self.create_reports_button)

        # Status messages table
        self.status_table = QTableWidget()
        self.status_table.setColumnCount(1)
        self.status_table.setHorizontalHeaderLabels(['Status Messages'])
        self.status_table.verticalHeader().setVisible(False)
        self.status_table.setEditTriggers(QAbstractItemView.NoEditTriggers)
        # Set font style for status messages
        self.status_table.setStyleSheet('''
            QTableWidget {
                background-color: #f0f0f0;
            }
            QTableWidget::item {
                font-family: "Courier New";
                font-size: 12px;
            }
        ''')
        main_layout.addWidget(self.status_table)

        self.setLayout(main_layout)

    def add_file(self, filepath):
        # Avoid adding duplicates
        existing_files = [self.file_table.item(row, 1).text() for row in range(self.file_table.rowCount())]
        if filepath in existing_files:
            return

        # Add a new row to the table with dummy data
        row_position = self.file_table.rowCount()
        self.file_table.insertRow(row_position)

        # Index
        index_item = QTableWidgetItem(str(row + 1))
        index_item.setTextAlignment(Qt.AlignCenter)
        self.file_table.setItem(row_position, 0, index_item)

        # File Path
        file_item = QTableWidgetItem(filepath)
        self.file_table.setItem(row_position, 1, file_item)

        # Dummy data for Manufacturer and Quarter
        manufacturer_item = QTableWidgetItem('Manufacturer {}'.format(row_position + 1))
        self.file_table.setItem(row_position, 2, manufacturer_item)

        quarter_item = QTableWidgetItem('Q{}'.format((row_position % 4) + 1))
        self.file_table.setItem(row_position, 3, quarter_item)

        # Checkboxes
        checkbox1 = QCheckBox()
        checkbox2 = QCheckBox()
        checkbox_layout1 = QHBoxLayout()
        checkbox_layout1.addWidget(checkbox1)
        checkbox_layout1.setAlignment(Qt.AlignCenter)
        checkbox_layout1.setContentsMargins(0, 0, 0, 0)
        checkbox_widget1 = QWidget()
        checkbox_widget1.setLayout(checkbox_layout1)
        self.file_table.setCellWidget(row_position, 4, checkbox_widget1)

        checkbox_layout2 = QHBoxLayout()
        checkbox_layout2.addWidget(checkbox2)
        checkbox_layout2.setAlignment(Qt.AlignCenter)
        checkbox_layout2.setContentsMargins(0, 0, 0, 0)
        checkbox_widget2 = QWidget()
        checkbox_widget2.setLayout(checkbox_layout2)
        self.file_table.setCellWidget(row_position, 5, checkbox_widget2)

        # Update Indexes
        self.update_indexes()

    def update_indexes(self):
        for row in range(self.file_table.rowCount()):
            index_item = QTableWidgetItem(str(row + 1))
            index_item.setTextAlignment(Qt.AlignCenter)
            self.file_table.setItem(row, 0, index_item)

    def toggle_always_on_top(self):
        if self.pin_button.isChecked():
            self.setWindowFlags(self.windowFlags() | Qt.WindowStaysOnTopHint)
            self.show()

            # macOS-specific code to make the window appear on all spaces
            if platform.system() == 'Darwin' and NSApp:
                # Activate the application
                NSApp.activateIgnoringOtherApps_(True)
                # Get the NSWindow instance
                nswindow = self.winId().__int__()
                from ctypes import c_void_p
                objc_id = c_void_p(nswindow)
                ns_window = objc.objc_object(objc_id)
                # Set the collection behavior
                ns_window.setCollectionBehavior_(NSWindowCollectionBehaviorCanJoinAllSpaces)
        else:
            self.setWindowFlags(self.windowFlags() & ~Qt.WindowStaysOnTopHint)
            self.show()
            # macOS-specific code to remove window from all spaces
            if platform.system() == 'Darwin' and NSApp:
                nswindow = self.winId().__int__()
                from ctypes import c_void_p
                objc_id = c_void_p(nswindow)
                ns_window = objc.objc_object(objc_id)
                ns_window.setCollectionBehavior_(0)

    def create_reports(self):
        # Placeholder for creating reports
        files = []
        for row in range(self.file_table.rowCount()):
            file_path = self.file_table.item(row, 1).text()
            files.append(file_path)
        # Update status messages with sample data
        self.status_table.setRowCount(0)
        for i, file in enumerate(files):
            self.add_status_message('Processing file {}: {}'.format(i+1, os.path.basename(file)))
            self.add_status_message('Success: Report generated for {}'.format(os.path.basename(file)))

    def add_status_message(self, message):
        row_position = self.status_table.rowCount()
        self.status_table.insertRow(row_position)
        message_item = QTableWidgetItem(message)
        # Set different text colors based on message content
        if 'Success' in message:
            message_item.setForeground(QColor('green'))
        elif 'Error' in message:
            message_item.setForeground(QColor('red'))
        else:
            message_item.setForeground(QColor('black'))
        self.status_table.setItem(row_position, 0, message_item)

    def close_application(self):
        reply = QMessageBox.question(
            self, 'Confirm Exit',
            'Are you sure you want to exit?',
            QMessageBox.Yes | QMessageBox.No, QMessageBox.No
        )
        if reply == QMessageBox.Yes:
            QApplication.instance().quit()

def main():
    app = QApplication(sys.argv)
    window = MainWindow()
    window.resize(800, 600)
    window.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

2024-11-27 22:00:57.959 Python[57498:26696452] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-27 22:00:57.959 Python[57498:26696452] +[IMKInputSession subclass]: chose IMKInputSession_Modern


NameError: name 'row' is not defined

AttributeError: 'NoneType' object has no attribute 'text'

AttributeError: 'NoneType' object has no attribute 'text'

AttributeError: 'NoneType' object has no attribute 'text'

SystemExit: 0

/Users/andreas/Lokal/CBAM - Lokal/Workspaces/ce_xml_converter/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
